In [11]:
from keras.applications import VGG16
#Load the VGG model
vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [12]:
from keras.preprocessing.image import ImageDataGenerator

In [13]:
for layer in vgg_conv.layers[:-4]:
    layer.trainable = False
 
# Check the trainable status of the individual layers
for layer in vgg_conv.layers:
    print(layer, layer.trainable)

<keras.engine.input_layer.InputLayer object at 0x000001B8A0B45488> False
<keras.layers.convolutional.Conv2D object at 0x000001B897665EC8> False
<keras.layers.convolutional.Conv2D object at 0x000001B8A0B3AE48> False
<keras.layers.pooling.MaxPooling2D object at 0x000001B899A57808> False
<keras.layers.convolutional.Conv2D object at 0x000001B8D1E0BE48> False
<keras.layers.convolutional.Conv2D object at 0x000001B8D1FB1588> False
<keras.layers.pooling.MaxPooling2D object at 0x000001B8D1FD0988> False
<keras.layers.convolutional.Conv2D object at 0x000001B8D200A288> False
<keras.layers.convolutional.Conv2D object at 0x000001B8D1FF5908> False
<keras.layers.convolutional.Conv2D object at 0x000001B8D203BA08> False
<keras.layers.pooling.MaxPooling2D object at 0x000001B8D206B548> False
<keras.layers.convolutional.Conv2D object at 0x000001B8D208E2C8> False
<keras.layers.convolutional.Conv2D object at 0x000001B8D20AFDC8> False
<keras.layers.convolutional.Conv2D object at 0x000001B8D20D15C8> False
<ker

In [24]:
from keras import models
from keras import layers
from keras import optimizers
 
# Create the model
model = models.Sequential()
 
# Add the vgg convolutional base model
model.add(vgg_conv)
 
# Add new layers
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(4, activation='softmax'))
 
# Show a summary of the model. Check the number of trainable parameters
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_2 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 1024)              25691136  
_________________________________________________________________
dense_5 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 4)                 2052      
Total params: 40,932,676
Trainable params: 33,297,412
Non-trainable params: 7,635,264
__________________________________

In [19]:
train_dir = 'Dataset/train'
validation_dir = 'Dataset/train'

In [20]:
import numpy as np
from random import shuffle

In [21]:
train_datagen = ImageDataGenerator(
      rescale=1./255,
#       rotation_range=20,
#       width_shift_range=0.2,
#       height_shift_range=0.2,
#       horizontal_flip=True,
#       fill_mode='nearest'
)
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# Change the batchsize according to your system RAM
train_batchsize = 16
val_batchsize = 10
 
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(224, 224),
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(224, 224),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=True)

Found 5121 images belonging to 4 classes.
Found 5121 images belonging to 4 classes.


In [22]:
print(train_generator.class_indices)

{'MildDemented': 0, 'ModerateDemented': 1, 'NonDemented': 2, 'VeryMildDemented': 3}


In [25]:
# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])
# Train the model
history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=3,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1)

Epoch 1/3
321/320 [==============================] - 6333s 20s/step - loss: 0.9532 - acc: 0.5448 - val_loss: 1.0022 - val_acc: 0.6676
Epoch 2/3
321/320 [==============================] - 6144s 19s/step - loss: 0.7124 - acc: 0.6911 - val_loss: 0.0928 - val_acc: 0.8088
Epoch 3/3
321/320 [==============================] - 6214s 19s/step - loss: 0.4678 - acc: 0.8147 - val_loss: 0.0086 - val_acc: 0.8282


In [27]:
#import cv2

In [28]:
"""img3 = cv2.imread('.<path to image test>')
img3 = cv2.resize(img3,(224,224))
img3 = np.reshape(img3,[1,224,224,3])"""

"img3 = cv2.imread('.<path to image test>')\nimg3 = cv2.resize(img3,(224,224))\nimg3 = np.reshape(img3,[1,224,224,3])"

In [29]:
#disType = model.predict_classes(img3)
#print(disType)

In [30]:
model.save('model_final.h5')